### Edgar Moises Hernandez-Gonzalez
#### 29-04-20-26-08-20
#### Clasificacion de imagenes motoras en EEG con CNN-1D + LSTM
#### Cargar modelo para evaluacion y predecir
##### Caracteristicas = Escalogramas CWT (3D)
##### Entrada resize proporcional de 495x500

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import numpy as np
import pandas as pd
import time
import pywt
import cv2
# from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import cohen_kappa_score, confusion_matrix
# from keras.utils import to_categorical
from keras.models import load_model

Using TensorFlow backend.


In [2]:
x_train = pd.read_csv("../Datos/MI-EEG-A09T.csv", header=None)
x_test = pd.read_csv("../Datos/MI-EEG-A09E.csv", header=None)
y_test = pd.read_csv("../Datos/EtiquetasA09E.csv", header=None)

In [3]:
print(x_train.shape)
print(x_test.shape)
print(y_test.shape)

(288, 22000)
(288, 22000)
(288, 1)


In [4]:
# esto se podria calcular asi n_clases = len(np.unique(y)), la y puede ser train o test
n_classes = len(np.unique(y_test))

print("n_classes:", n_classes)

n_classes: 4


In [ ]:
# codificar etiquetas con valores de 0 a nClases-1
# y_test = LabelEncoder().fit_transform(y_test)

In [5]:
#unir 22 canales
def unir_escalogramas_vertical(data):
  dim = (500, 495) #ancho, alto
  fs = 250
  escalas = pywt.scale2frequency('cmor3-3', np.arange(8,30.5,0.5)) / (1/fs)
  datos = np.zeros((data.shape[0],495,500))
  temporal = np.zeros((990,1000))
  for i in range(data.shape[0]): #288 muestras
    for j in range(22): #22 canales
      sig = data.iloc[i, j*1000:(j+1)*1000]
      #sig = sig[124:624]
      # escalograma CWT
      coef, freqs = pywt.cwt(sig, escalas, 'cmor3-3', sampling_period = (1 / fs))
      temporal[j*45:(j+1)*45, :] = abs(coef)
    resized = cv2.resize(temporal, dim, interpolation=cv2.INTER_AREA)
    datos[i] = resized
    if i % 100 == 0: #esto solo es para ver como avanza
      print(i)
  return datos

In [6]:
#llamar a unir_escalogramas_vertical(data)
inicio = time.time()

x_train = unir_escalogramas_vertical(x_train)
x_test = unir_escalogramas_vertical(x_test)

fin = time.time()
print("Tiempo:", fin - inicio)

0
100
200
0
100
200
Tiempo: 193.38124012947083


In [7]:
print(x_test.shape)

(288, 495, 500)


In [8]:
x = []
x.append(np.ceil(np.max(x_train)))
# esta linea solo se ocupa si se va a predecir un modelo creado con CV
# x.append(np.ceil(np.max(x_test)))

In [9]:
#convertir a float
x_test = x_test.astype('float32')

#escalar los valores en un rango de 0 a 1 (normalizar)
x_test /= np.ceil(np.max(x))

In [10]:
#convertir de 3D a 5D
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1], x_test.shape[2], 1))

print(x_test.shape)

(288, 1, 495, 500, 1)


In [11]:
model = load_model('../Modelos/RNN_CWT09.hdf5')

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 1, 495, 500, 16)   160       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 1, 247, 250, 16)   0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 1, 247, 250, 16)   2320      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 1, 123, 125, 16)   0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 1, 246000)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                31492224  
_________________________________________________________________
dense_1 (Dense)              (None, 256)              

In [15]:
inicio = time.time()

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

fin = time.time()
print("Tiempo:", fin - inicio)

Tiempo: 1.480736494064331


In [ ]:
print("loss: %.4f" % test_loss)
print("accuracy: %.4f" % test_acc)

loss: 0.2377
accuracy: 0.9236


In [ ]:
probabilidades = model.predict(x_test)
y_pred = np.argmax(probabilidades, 1) #1=fila

In [ ]:
kappa = cohen_kappa_score(y_test, y_pred)
print("kappa: %.4f " % kappa)

kappa: 0.8472 


In [ ]:
matriz_confusion = confusion_matrix(y_test, y_pred)
print("confusion matrix:\n", matriz_confusion)

confusion matrix:
 [[69  3]
 [ 8 64]]


In [16]:
inicio = time.time()

probabilidades = model.predict(x_test[0].reshape(1,1,495,500,1))
y_pred = np.argmax(probabilidades, 1) #1=fila

fin = time.time()
print("Tiempo:", fin - inicio)

Tiempo: 0.026706218719482422
